<a href="https://colab.research.google.com/github/Prashanna-Raj-Pandit/Transfer-Learning--TensorFlow/blob/main/Transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf

In [2]:
dataset, info = tfds.load("tf_flowers",
                          as_supervised=True,
                          with_info=True)

In [14]:
info

tfds.core.DatasetInfo(
    name='tf_flowers',
    full_name='tf_flowers/3.0.1',
    description="""
    A large set of images of flowers
    """,
    homepage='https://www.tensorflow.org/tutorials/load_data/images',
    data_dir='/root/tensorflow_datasets/tf_flowers/3.0.1',
    file_format=tfrecord,
    download_size=218.21 MiB,
    dataset_size=221.83 MiB,
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=5),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'train': <SplitInfo num_examples=3670, num_shards=2>,
    },
    citation="""@ONLINE {tfflowers,
    author = "The TensorFlow Team",
    title = "Flowers",
    month = "jan",
    year = "2019",
    url = "http://download.tensorflow.org/example_images/flower_photos.tgz" }""",
)

In [15]:
dataset_size = info.splits["train"].num_examples
class_names = info.features["label"].names
n_classes = info. features["label"].num_classes
dataset_size, class_names, n_classes

(3670, ['dandelion', 'daisy', 'tulips', 'sunflowers', 'roses'], 5)

In [5]:
test_set_raw, valid_set_raw, train_set_raw = tfds.load("tf_flowers",
                        split=["train[:10%]",
                               "train[10%:25%]",
                               "train[25%:]"],
                        as_supervised=True)

In [6]:
def preprocess(image,label):
    resized_image = tf.image.resize(image, [224,224])
    final_image = tf.keras.applications.xception.preprocess_input(resized_image) #t scales the pixel values of the input image to the range of -1 to 1.
    return final_image, label


In [18]:

train_set = train_set_raw.shuffle(1000)
train_set

<_ShuffleDataset element_spec=(TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [16]:
batch_size = 32
train_set = train_set_raw.shuffle(1000)
train_set = train_set.map(preprocess).batch(batch_size).prefetch(1)
valid_set = valid_set_raw.map(preprocess).batch(batch_size).prefetch(1)
test_set = test_set_raw.map(preprocess).batch(batch_size).prefetch(1)

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [8]:
base_model = tf.keras.applications.xception.Xception(weights="imagenet",
                                            include_top=False)


avg = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
output = tf.keras.layers.Dense(n_classes, activation="softmax")(avg)


model = tf.keras.Model(inputs=base_model.input, outputs=output)

In [9]:
for layer in base_model.layers:
  layer.trainable = False

In [10]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.2,
                                    momentum=0.9,
                                    decay=0.01)

model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])


history = model.fit(train_set,
                    epochs=5,
                    validation_data=valid_set)

Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


86/86 ━━━━━━━━━━━━━━━━━━━━ 33s 239ms/step - accuracy: 0.7274 - loss: 1.3830 - val_accuracy: 0.8294 - val_loss: 1.1597
Epoch 2/5
86/86 ━━━━━━━━━━━━━━━━━━━━ 25s 154ms/step - accuracy: 0.8667 - loss: 0.8276 - val_accuracy: 0.8348 - val_loss: 1.5091
Epoch 3/5
86/86 ━━━━━━━━━━━━━━━━━━━━ 21s 160ms/step - accuracy: 0.8941 - loss: 0.6497 - val_accuracy: 0.8475 - val_loss: 1.3049
Epoch 4/5
86/86 ━━━━━━━━━━━━━━━━━━━━ 20s 154ms/step - accuracy: 0.9347 - loss: 0.3497 - val_accuracy: 0.8730 - val_loss: 1.2599
Epoch 5/5
86/86 ━━━━━━━━━━━━━━━━━━━━ 21s 155ms/step - accuracy: 0.9527 - loss: 0.3094 - val_accuracy: 0.8530 - val_loss: 1.3203


In [11]:
for layer in base_model.layers:
  layer.trainable = True

In [12]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01,
                                    momentum=0.9,
                                    decay=0.001)

model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

history = model.fit(train_set,
                    epochs=10,
                    validation_data=valid_set)

Epoch 1/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 79s 531ms/step - accuracy: 0.8110 - loss: 0.7370 - val_accuracy: 0.8040 - val_loss: 1.3182
Epoch 2/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 58s 477ms/step - accuracy: 0.9566 - loss: 0.1360 - val_accuracy: 0.8893 - val_loss: 0.3512
Epoch 3/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 41s 475ms/step - accuracy: 0.9856 - loss: 0.0485 - val_accuracy: 0.8911 - val_loss: 0.3549
Epoch 4/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 40s 469ms/step - accuracy: 0.9913 - loss: 0.0308 - val_accuracy: 0.9002 - val_loss: 0.3351
Epoch 5/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 42s 478ms/step - accuracy: 0.9954 - loss: 0.0211 - val_accuracy: 0.9056 - val_loss: 0.4006
Epoch 6/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 82s 478ms/step - accuracy: 0.9954 - loss: 0.0166 - val_accuracy: 0.9220 - val_loss: 0.2650
Epoch 7/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 82s 476ms/step - accuracy: 0.9971 - loss: 0.0072 - val_accuracy: 0.9220 - val_loss: 0.2990
Epoch 8/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 82s 477ms/step - accuracy: 0.9950 - loss: 0.0103 - val_accu